# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [3]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [40]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***@MongoStrobil***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key): vtqB0NkNxZDKfQrTePLiRyibs
    1. Consumer Secret (API Secret): RfaASnTNAEPP8V81PTtKNwLH1SzSpYXuVEiPNZMEjnu1CoOvtE
1. Mais abaixo, gere um Token e anote também:
    1. Access Token: 905108217595822080GaArhSqsnTiHg0BEjJbnRdnDebVi7qx
    1. Access Token Secret: ED7G6dt1dqHvxNgxrf5J9yFuUBoGiQ5cYNwoScFfiCe3b
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [5]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [6]:
#Produto escolhido:
produto = 'Got'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [7]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [8]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

In [12]:
Got = pd.read_excel("Game of Thrones.xlsx", encode="UTF-8")

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [17]:
for e in range(0,300):
    sub = Got.Treinamento[e]
    sub = sub.replace("\n", ' ')
    sub = sub.replace(":", ' ')
    sub = sub.replace('"', ' ')
    sub = sub.replace("'", ' ')
    sub = sub.replace('(', ' ')
    sub = sub.replace(')', ' ')
    sub = sub.replace(',', ' ')
    sub = sub.replace('+', ' ')
    sub = sub.replace('/', ' ')
    sub = sub.replace('-', ' ')
    sub = sub.replace('@', ' ')
    sub = sub.replace('!', ' ')
    sub = sub.replace('?', ' ')
    sub = sub.replace('#', ' ')
    sub = sub.split()
    sub = " ".join(sub)
    Got.Treinamento[e] = sub

In [63]:
from pandas import *

def prob(li_frase,li_lista):
    total_pal = 0
    for item in range(0,len(li_lista)):
        total_pal += len(li_lista[item])
    
    for palavra in li_frase:
        contador = 0
        
        for frase in li_lista:
            for palavrinha in frase:
                if palavrinha == palavra:
                    contador += 1
        
        exec(str(palavra)+'_prob = '+str(contador/total_pal))
        
        
        

a = Got.loc[Got.Relevância=="Relevante",['Treinamento']]

total_pal_rel = 0

lista_frases = a["Treinamento"].tolist()
Relevantes = [item.split() for item in lista_frases]

for item in range(0,len(Relevantes)):
    total_pal_rel += len(Relevantes[item])

print(Relevantes)
print(total_pal_rel)

[['rt', 'marciorcm', 'gostei', 'de', 'um', 'vídeo', 'youtube', 'https', 't.co', 'wu5y0mmuef', 'dos', 'starks', 'e', 'seus', 'significados', 'game', 'of', 'thrones', 'explicado'], ['gente', 'cenários', 'reais', 'de', 'game', 'of', 'thrones', 'que', 'você', 'pode', 'visitar', 'https', 't.co', '25hlllqvsy', 'via', 'shutterstockpt'], ['5', 'séries', 'da', 'hbo', 'para', 'você', 'ver', 'antes', 'do', 'retorno', 'de', 'game', 'of', 'thrones', 'https', 't.co', 'rxf8kkrh1r', 'https', 't.co', 'uc6rhrogji'], ['rt', 'marciorcm', 'gostei', 'de', 'um', 'vídeo', 'youtube', 'https', 't.co', 'qnnodh7eog', 'da', 'genética', 'nas', 'famílias', 'de', 'game', 'of', 'thrones', 'stark', 'e', 'targaryen'], ['eu', 'nao', 'consigo', 'comer', 'e', 'ver', 'game', 'of', 'thrones', 'ao', 'mesmo', 'tempo'], ['rt', 'gfiuza_oficial', 'game', 'of', 'thrones', 'acabou', 'mas', 'toda', 'semana', 'temos', 'ana', 'paula', 'destronando', 'geral', 'os', 'canastrões.', 'https', 't.co', 'rclo6valxp'], ['rt', 'poeticou', 'o', 

___-
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
